# Convolutional Neural network for soybean diseases identification

Based and adapted from https://github.com/nzb0054/RoboCrop-CNN-WebApp/blob/main/robocrop_cnn.py

In [1]:
import numpy as np
import csv
import os
import random
import tensorflow as tf
import itertools
from tensorflow import keras
from tensorflow.keras import layers, regularizers, optimizers
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
# Need to edit below import if using base model other than DenseNet
from tensorflow.keras.applications.densenet import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.optimizers import Adam, SGD
# Need to edit below import if using base model other than DenseNet
from tensorflow.keras.applications.densenet import DenseNet201
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from keras.applications.xception import Xception


In [2]:
load_trained = True  # Carrega o modelo salvo no último treinamento
save_after_each_epoch = True  # Salva o modelo a cada epoch concluída
epochs = 30  # Quantidade de epochs do treinamento
# Modelo base a ser utilizado. DenseNet201, ResNet50, Xception ou EfficientNetB0
base_model_name = "ResNet50"

In [3]:
# Setting seeds so that randomization is kept consistent.
np.random.seed(70)
random.seed(70)
tf.random.set_seed(70)

models = {
    "EfficientNetB0": EfficientNetB0,
    "ResNet50": ResNet50,
    "DenseNet201": DenseNet201,
    "Xception": Xception
}

# This is the dimensions that each image will be shaped to
# DenseNet201 requires 224 x 224.
img_height, img_width = (224, 224)
# Batch size is the number of training examples utilized in one iteration. Could use 16, may increase compute time.
batch_size = 32

# These are the directories/folders where your images are stored for training, validation, and test datasets.
train_data_dir = "dataset/train"
valid_data_dir = "dataset/test"
test_data_dir = "dataset/val"

# ImageDataGenerator performs image augmentation for each image on the fly. Rotating, flipping, brightness, etc.
train_datagen = ImageDataGenerator(
    width_shift_range=0.2,  # 0.2 fraction of total width/height
    height_shift_range=0.2,
    fill_mode="nearest",
    brightness_range=[0.9, 1.1],  # range for picking a shift value from
    rotation_range=30,  # degree range for random rotations
    vertical_flip=True,
    horizontal_flip=True,
    validation_split=0.0,
    rescale=1./255)  # rescaling image pixel values by the number of channels, 1/255

# Pulls your training dataset images
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'  # more than 2 classes --> binary
)

# Pulls your validation dataset images
valid_generator = train_datagen.flow_from_directory(
    valid_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

# Pulls your test dataset images. Note that you only want to use 1 image at a time for test.
test_generator = train_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=1,
    class_mode='categorical')


if load_trained:
    # load previous trained model
    model = keras.models.load_model(
        f"saved_models/{base_model_name}/dense_1118.h5")
else:
    # Sets base model architecture, could swap out DenseNet201 for ResNet50, VGG16, etc
    # Whether to include the fully-connected layer at top of network
    # 'imagenet' (pre-training on ImageNet), or the path to the weights file to be loaded.
    base_model = models[base_model_name](include_top=False, weights='imagenet')
    # Below x lines are the additional architecture attached beyond the base model
    x = base_model.output
    # Globalaveragepooling performs the 'flatten' purposes
    x = GlobalAveragePooling2D()(x)
    # 128, 64 neuron fully-connected layers with relu activation
    # Dropout for regularization
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.3)(x)
    # Predicting a label for each image based on softmax activation, for 9 classes
    predictions = Dense(train_generator.num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    # Deciding whether to freeze the base model weights (imagenet) or allow them to update
    # Setting this to 'True' implies we are training the entire model
    for layer in base_model.layers:
        layer.trainable = True

    # Compile the model using stochastic gradient descent w/ learning rate and momentum values
    # Use categorical crossentropy as the loss function
    model.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9),
                  loss='categorical_crossentropy', metrics=['accuracy'])

# CSV logger automatically keeps track of accuracy and loss for both training and validation during each epoch.
# This is very convenient for graphing model performance through time later in R
# Outputs a .csv file
log_csv = CSVLogger(
    f"saved_models/{base_model_name}/metrics.csv", separator=',', append=True)
callbacks_list = [log_csv]

# Pega o maior valor de acurácia de validação do arquivo de métricas


def get_max_val_accuracy(file_path):
    max_accuracy = 0
    with open(file_path, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            accuracy = float(row['val_accuracy'])
            if accuracy > max_accuracy:
                max_accuracy = accuracy
    return max_accuracy


if save_after_each_epoch:
    # Callback de checkpoint para salvar o modelo após cada época
    model_checkpoint_callback = ModelCheckpoint(
        filepath=f'saved_models/{base_model_name}/dense_1118.h5',
        save_weights_only=False,
        verbose=1)

    # Valor inicial de acurácia de validação para o callback de checkpoint de melhor acurácia
    initial_value_threshold = get_max_val_accuracy(
        f"saved_models/{base_model_name}/metrics.csv") if os.path.exists(f"saved_models/{base_model_name}/metrics.csv") else 0

    # Callback de checkpoint para salvar o modelo com a melhor acurácia de validação
    model_checkpoint_callback_best = ModelCheckpoint(
        filepath=f'saved_models/{base_model_name}/best_val_acc.h5',
        save_weights_only=False,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        initial_value_threshold=initial_value_threshold,
        verbose=1)

    callbacks_list.append(model_checkpoint_callback)
    callbacks_list.append(model_checkpoint_callback_best)

# Train the model, verbose just tells you much info to display during training
model.fit(train_generator, epochs=epochs, verbose=1,
          validation_data=valid_generator, callbacks=callbacks_list)

# Save the model and weights
# Mainly need the .h5 file to best hosted on the webapp
model.save_weights(f'saved_models/{base_model_name}/dense_weights_1118')
model.save(f'saved_models/{base_model_name}/dense_1118.hdf5')
if not save_after_each_epoch:
    model.save(f'saved_models/{base_model_name}/dense_1118.h5')


Found 2412 images belonging to 9 classes.
Found 819 images belonging to 9 classes.
Found 801 images belonging to 9 classes.
Epoch 1/30
76/76 [==============================] - ETA: 0s - loss: 0.4985 - accuracy: 0.8611 
Epoch 1: saving model to saved_models/ResNet50\dense_1118.h5

Epoch 1: val_accuracy improved from 0.83272 to 0.84005, saving model to saved_models/ResNet50\best_val_acc.h5
76/76 [==============================] - 1515s 20s/step - loss: 0.4985 - accuracy: 0.8611 - val_loss: 0.5272 - val_accuracy: 0.8400
Epoch 2/30
 1/76 [..............................] - ETA: 26:29 - loss: 0.3832 - accuracy: 0.9062

KeyboardInterrupt: 